<a href="https://colab.research.google.com/github/KARENCMP82/Python/blob/main/3_KMEANS_24MAR25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Nuclio Digital School - Máster en Data Science**

# Unsupervised Learning: K-MEANS

# *Profesora: Raquel Revilla*

# Objetivos del notebook

En este notebook vamos a trabajar con el dataset de la **compañia brasileña de E-Commerce llamada Olist.**

La compañia opera con una filosofía de ***SaaS (Software as a Service)***. Su producto básico es un marketplace donde tiendas **con o sin presencia online pueden vender sus productos a los clientes que visitan su web (una especie de Amazon en Brasil).**

![Data Model](https://i.imgur.com/HRhd2Y0.png)

Utilizando el dataset de Olist (alrededor de 100 mil registros) vamos a construir una segmentacion basada en el algoritmo KMeans. El enfoque a la hora de construir variables estará basado en el modelo de ***RFM (recency - frequency - monetary value).*** Esta es una forma de trabajar muy común en startups y tiendas online donde se dispone de poca información de los clientes y la retención/canje de los clicks es fundamental.

El modelo RFM es una técnica de análisis de datos utilizada en marketing para evaluar y segmentar la base de clientes de una empresa. RFM es un acrónimo que representa las tres dimensiones clave utilizadas en el análisis:

*   Recency (Recencia): Mide el tiempo transcurrido desde la última compra del cliente. Los clientes que han comprado recientemente son más propensos a responder a nuevas ofertas que aquellos que no han comprado en mucho tiempo.
*   Frequency (Frecuencia): Mide el número de veces que un cliente ha realizado una compra durante un período determinado. Los clientes que compran con mayor frecuencia tienden a ser más leales y valiosos.
*   Monetary (Valor Monetario): Mide el valor total de las compras realizadas por un cliente en un período determinado. Los clientes que gastan más dinero son considerados más valiosos para la empresa.

Nuestros principales objetivos serán:
1. **Construir variables de negocio (pensadas para nuestra segmentación)**.

2. Usar el diagrama del codo para determinar la cantidad **"óptima"** (**CORE IDEA**) de centroides para el modelo de KMeans.

3. **Resumir la información de nuestros clústers en un formato más amigable** usando Pandas (fichas de clientes).

Antes de seguir con el notebook, vamos a ver el funcionamiento de KMeans de manera [visual](https://www.naftaliharris.com/blog/visualizing-k-means-clustering/) para comprender como aprende exactamente y también su relación con la distancia euclídea.

<a id = "table_of_contents"></a>
# Índice

[Importación de las principales librerías](#import_modules)

[Importación de los datos](#import_data)

[Exploratory Data Analysis (EDA)](#eda)

---> [EDA customers df](#df1)

---> [EDA orders df](#df2)

---> [EDA payments df](#df3)

[Creación de variables](#fe)

[Join final con clientes y variables finales](#join)

[Creación de nuestros propios Transformers (CORE IDEA)](#skpipeline)

[Elbow curve (CORE IDEA)](#elbow_curve)

[Segmentación de los clientes con la "k adecuada" (CORE IDEA)](#segmentacion)

[Ficha de clientes](#ficha)

[Conclusión](#conclusión)

<a id = "import_modules"></a>
# Importación de las principales librerías
[Volver al índice](#table_of_contents)

En esta sección del kernel vamos a cargar las principales librerías que vamos a usar en nuestro notebook durante la implementación del algoritmo **KMeans.**

In [58]:
# silence warnings
import warnings
warnings.filterwarnings("ignore")

# operating system
import os

# time calculation to track some processes
import time

# numeric and matrix operations
import numpy as np
import pandas as pd

# loading ploting libraries
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# python core library for machine learning and data science
import sklearn
from sklearn import set_config
set_config(transform_output = "pandas")

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import RobustScaler, MinMaxScaler
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.cluster import KMeans

In [59]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [60]:
#RANDOM_STATE = 175
# Define una variable llamada 'RANDOM_STATE' y le asigna el valor 175.
# Este valor se utiliza como semilla para los generadores de números aleatorios en diferentes partes del código.
# Establecer una semilla permite reproducir los mismos resultados cada vez que se ejecuta el código, lo que es útil para la reproducibilidad de los experimentos.

#PATH_FOLDER = '/content/drive/MyDrive/Nuclio_No_Supervisado/unsupervised_learning/data/'
# Define una variable llamada 'PATH_FOLDER' y le asigna la ruta de la carpeta que contiene los datos.
# Esta variable se utiliza para facilitar el acceso a los archivos de datos en el código, evitando tener que escribir la ruta completa cada vez que se necesita acceder a un archivo.

In [61]:
print("Working with this sklearn version {}".format(sklearn.__version__))

Working with this sklearn version 1.6.1


<a id = "import_data"></a>
# Importación de los datos
[Volver al índice](#table_of_contents)

En la presente sección del kernel vamos a cargar los principales datasets que vamos a usar para nuestra segmentación.

El alumno puede **añadir otros datasets o datos externos** para profundizar o experimentar con el algoritmo KMeans.

In [62]:
PATH_CUSTOMERS = "/content/drive/MyDrive/Colab Notebooks/Ejercicios/kmeans_olist_customers_dataset.csv"  # Guarda la RUTA del archivo, no el DataFrame

customer_df = pd.read_csv(PATH_CUSTOMERS) # Lee el archivo con la ruta

In [63]:
customer_df.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


In [64]:
PATH_FOLDER = '/content/drive/MyDrive/Colab Notebooks/Ejercicios/'  # Define la carpeta principal de los datos.

PATH_ORDERS = os.path.join(PATH_FOLDER, 'kmeans_olist_orders_dataset.csv')  # Crea la ruta completa al archivo de órdenes.

orders_df = pd.read_csv(PATH_ORDERS)  # Lee el archivo de órdenes y lo guarda en un DataFrame.

In [65]:
orders_df.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


In [66]:
PATH_FOLDER = '/content/drive/MyDrive/Colab Notebooks/Ejercicios/'  # Tu ruta base (ya definida, no es necesario repetirla si ya la definiste antes)

PATH_PAYMENTS = os.path.join(PATH_FOLDER, 'kmeans_olist_order_payments_dataset.csv')

payments_df = pd.read_csv(PATH_PAYMENTS)

In [67]:
payments_df.head()

,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
4,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45


<a id = "eda"></a>
# Exploratory Data Analysis (EDA)
[Volver al índice](#table_of_contents)

En la sección del EDA haremos **una primera aproximación a nuestros datos** para ver su composición y que variables tenemos a nuestra disposición.

<a id = "df1"></a>
# EDA customers df
[Volver al índice](#table_of_contents)

EDA rápido sobre el **dataset de clientes.**

In [68]:
def report_df(df, verbose = True):
    '''
    Hace un report simple sobre el DataFrame suministrado.
    '''
    print(df.info(verbose = verbose))
    total_nulos = df.isnull().sum().sum()
    print()
    print(f"Tenemos un total de {total_nulos} nulos")

In [69]:
report_df(customer_df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               99441 non-null  object
 1   customer_unique_id        99441 non-null  object
 2   customer_zip_code_prefix  99441 non-null  int64 
 3   customer_city             99441 non-null  object
 4   customer_state            99441 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.8+ MB
None

Tenemos un total de 0 nulos


In [70]:
customer_df.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


In [71]:
customer_id = customer_df['customer_id'].nunique()
customer_unique_id = customer_df['customer_unique_id'].nunique()

print(f'''Tenemos un total de {customer_unique_id} clientes únicos,\n
para un total de {customer_id} pedidos (en el esquema de Olist, especifican que el id único es customer_unique_id\n
y que customer_id es un id que se genera en cada compra y por tanto a la práctica viene a ser lo mismo que un pedido).\n
Esto implica un ratio de {round(customer_id/customer_unique_id, 2)} pedidos por cliente.
''')

Tenemos un total de 96096 clientes únicos,

para un total de 99441 pedidos (en el esquema de Olist, especifican que el id único es customer_unique_id

y que customer_id es un id que se genera en cada compra y por tanto a la práctica viene a ser lo mismo que un pedido).

Esto implica un ratio de 1.03 pedidos por cliente.



<a id = "df2"></a>
# EDA orders df
[Volver al índice](#table_of_contents)

EDA rápido sobre el **dataset de pedidos.**

In [72]:
report_df(orders_df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   order_id                       99441 non-null  object
 1   customer_id                    99441 non-null  object
 2   order_status                   99441 non-null  object
 3   order_purchase_timestamp       99441 non-null  object
 4   order_approved_at              99281 non-null  object
 5   order_delivered_carrier_date   97658 non-null  object
 6   order_delivered_customer_date  96476 non-null  object
 7   order_estimated_delivery_date  99441 non-null  object
dtypes: object(8)
memory usage: 6.1+ MB
None

Tenemos un total de 4908 nulos


In [73]:
orders_df.head(3).T
 # Muestra las 3 primeras filas del DataFrame 'orders_df' y luego lo transpone (intercambia filas por columnas) para facilitar la visualización.

,0,1,2
order_id,e481f51cbdc54678b7cc49136f2d6af7,53cdb2fc8bc7dce0b6741e2150273451,47770eb9100c2d0c44946d9cf07ec65d
customer_id,9ef432eb6251297304e76186b10a928d,b0830fb4747a6c6d20dea0b8c802d7ef,41ce2a54c0b03bf3443c3d931a367089
order_status,delivered,delivered,delivered
order_purchase_timestamp,2017-10-02 10:56:33,2018-07-24 20:41:37,2018-08-08 08:38:49
order_approved_at,2017-10-02 11:07:15,2018-07-26 03:24:27,2018-08-08 08:55:23
order_delivered_carrier_date,2017-10-04 19:55:00,2018-07-26 14:31:00,2018-08-08 13:50:00
order_delivered_customer_date,2017-10-10 21:25:13,2018-08-07 15:27:45,2018-08-17 18:06:29
order_estimated_delivery_date,2017-10-18 00:00:00,2018-08-13 00:00:00,2018-09-04 00:00:00


In [74]:
orders_df['order_status'].value_counts()
# Cuenta la frecuencia de cada valor único en la columna 'order_status' del DataFrame 'orders_df'.  El resultado muestra cuántas órdenes hay en cada estado (ej: delivered, canceled, etc.).

,count
order_status,
delivered,96478
shipped,1107
canceled,625
unavailable,609
invoiced,314
processing,301
created,5
approved,2


In [75]:
orders_df.isnull().sum()

,0
order_id,0
customer_id,0
order_status,0
order_purchase_timestamp,0
order_approved_at,160
order_delivered_carrier_date,1783
order_delivered_customer_date,2965
order_estimated_delivery_date,0


In [76]:
order_id = orders_df['order_id'].nunique()
customer_unique_id = customer_df['customer_unique_id'].nunique()

print(f"Tenemos un total de {order_id} pedidos")
print(f"Tenemos un total de {customer_unique_id} customer únicos (de la tabla de customers)")
print(f"La relación de pedidos por clientes es de {round(order_id/customer_unique_id, 2)}")

Tenemos un total de 99441 pedidos
Tenemos un total de 96096 customer únicos (de la tabla de customers)
La relación de pedidos por clientes es de 1.03


<a id = "df3"></a>
# EDA payments df
[Volver al índice](#table_of_contents)

EDA rápido sobre el **dataset de pagos.**

In [77]:
report_df(payments_df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103886 entries, 0 to 103885
Data columns (total 5 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   order_id              103886 non-null  object 
 1   payment_sequential    103886 non-null  int64  
 2   payment_type          103886 non-null  object 
 3   payment_installments  103886 non-null  int64  
 4   payment_value         103886 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 4.0+ MB
None

Tenemos un total de 0 nulos


In [78]:
payments_df.head()

,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
4,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45


In [79]:
order_id_pay = payments_df['order_id'].nunique()

print(f"Tenemos un total de {order_id_pay} pedidos únicos en la tabla de payments")

Tenemos un total de 99440 pedidos únicos en la tabla de payments


In [80]:
payments_df['payment_type'].value_counts()

,count
payment_type,
credit_card,76795
boleto,19784
voucher,5775
debit_card,1529
not_defined,3


<a id = "fe"></a>
# Creación de variables
[Volver al índice](#table_of_contents)

Hemos analizado los 3 datasets claves con los que vamos a trabajar.

En esta sección del notebook, **iremos agregando el dataset y vamos a generar nuevas variables** para que después en la siguiente sección lo juntamos en uno y hacemos la segmentación.

**BRAIN STORMING**

*   número de compras por cliente en un período determinado
*   ticket medio por cliente
*   días desde la última compra por cliente
*   período con más compras por cliente
*   compras retrasos por cliente
*   devoluciones de compras por cliente
*   compra en fin de semana por cliente
*   compra entre semana por cliente
*   compra en horario laboral por cliente
*   compra no laboral por cliente
*   compra noche por cliente
*   máximo valor de una compra por cliente
*   mínimo valor de una compra por cliente
*   número de productos que se compran en un pedido por cliente
*   método de pago por cliente

In [81]:
aggregated_payments = payments_df.groupby('order_id').agg(
    max_pay = ('payment_value', 'max'),  # Calcula el pago máximo por orden.
    min_pay = ('payment_value', 'min'),  # Calcula el pago mínimo por orden.
    mean_pay = ('payment_value', 'mean'), # Calcula el pago promedio por orden.
    total_pay = ('payment_value', 'sum'), # Calcula el pago total por orden.
    max_seq = ('payment_sequential', 'max') # Calcula la secuencia de pago máxima por orden.
)

# Agrupa los pagos por 'order_id' y calcula varias estadísticas de pago para cada orden.

In [82]:
aggregated_payments.head()
#pagos agregados o pagos resumidos.

,max_pay,min_pay,mean_pay,total_pay,max_seq
order_id,,,,,
00010242fe8c5a6d1ba2dd792cb16214,72.19,72.19,72.19,72.19,1
00018f77f2f0320c557190d7a144bdd3,259.83,259.83,259.83,259.83,1
000229ec398224ef6ca0657da4fc703e,216.87,216.87,216.87,216.87,1
00024acbcdf0a6daa1e931b038114c75,25.78,25.78,25.78,25.78,1
00042b26cf59d7ce69dfabb4e55b4fd9,218.04,218.04,218.04,218.04,1


In [83]:
aggregated_payments[aggregated_payments['max_seq'] == 3].head()

,max_pay,min_pay,mean_pay,total_pay,max_seq
order_id,,,,,
00bd50cdd31bd22e9081e6e2d5b3577b,40.46,4.88,28.600000,85.80,3
039f61edec89c6f0edf8dd1a0bdea1fe,166.22,50.00,105.406667,316.22,3
03e5d6a8dd520898f86f8429ccd07c21,46.80,42.24,45.280000,135.84,3
06875ab72c5b9bb2eb303a70031bfeb0,22.68,2.32,15.893333,47.68,3
070d6fe21e7d454b11f1cb27ca2c15c0,89.46,4.78,39.410000,118.23,3


In [84]:
orders_with_payments = pd.merge(orders_df, aggregated_payments, on = 'order_id')

# Combina (merge) los DataFrames 'orders_df' y 'aggregated_payments' basándose en la columna común 'order_id'.
# El resultado es un nuevo DataFrame llamado 'orders_with_payments' que contiene información de ambos DataFrames, unida por el 'order_id'.

In [85]:
report_df(orders_with_payments)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99440 entries, 0 to 99439
Data columns (total 13 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   order_id                       99440 non-null  object 
 1   customer_id                    99440 non-null  object 
 2   order_status                   99440 non-null  object 
 3   order_purchase_timestamp       99440 non-null  object 
 4   order_approved_at              99280 non-null  object 
 5   order_delivered_carrier_date   97657 non-null  object 
 6   order_delivered_customer_date  96475 non-null  object 
 7   order_estimated_delivery_date  99440 non-null  object 
 8   max_pay                        99440 non-null  float64
 9   min_pay                        99440 non-null  float64
 10  mean_pay                       99440 non-null  float64
 11  total_pay                      99440 non-null  float64
 12  max_seq                        99440 non-null 

In [86]:
orders_with_payments.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,max_pay,min_pay,mean_pay,total_pay,max_seq
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,18.59,2.00,12.903333,38.71,3
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,141.46,141.46,141.460000,141.46,1
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,179.12,179.12,179.120000,179.12,1
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00,72.20,72.20,72.200000,72.20,1
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00,28.62,28.62,28.620000,28.62,1


In [87]:
orders_with_payments.set_index('order_id', inplace = True)
#nos pone de order id es nuestro indice

In [88]:
orders_with_payments.head()

,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,max_pay,min_pay,mean_pay,total_pay,max_seq
order_id,,,,,,,,,,,,
e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,18.59,2.00,12.903333,38.71,3
53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,141.46,141.46,141.460000,141.46,1
47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,179.12,179.12,179.120000,179.12,1
949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00,72.20,72.20,72.200000,72.20,1
ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00,28.62,28.62,28.620000,28.62,1


In [89]:
orders_with_payments['order_purchase_timestamp'] =\
pd.to_datetime(orders_with_payments['order_purchase_timestamp'], format = '%Y-%m-%d %H:%M:%S')
# Convierte la columna 'order_purchase_timestamp' del DataFrame 'orders_with_payments' al tipo de dato datetime.
# 'pd.to_datetime(...)': Función de pandas para convertir a datetime.
# 'format = '%Y-%m-%d %H:%M:%S'': Especifica el formato de la fecha y hora en la columna original.

orders_with_payments['order_delivered_customer_date'] =\
pd.to_datetime(orders_with_payments['order_delivered_customer_date'], format = '%Y-%m-%d %H:%M:%S')
# Convierte la columna 'order_delivered_customer_date' al tipo datetime, asumiendo el mismo formato.

orders_with_payments['order_estimated_delivery_date'] =\
pd.to_datetime(orders_with_payments['order_estimated_delivery_date'], format = '%Y-%m-%d %H:%M:%S')
# Convierte la columna 'order_estimated_delivery_date' al tipo datetime, asumiendo el mismo formato.

In [90]:
orders_with_payments.head()

,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,max_pay,min_pay,mean_pay,total_pay,max_seq
order_id,,,,,,,,,,,,
e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,18.59,2.00,12.903333,38.71,3
53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,141.46,141.46,141.460000,141.46,1
47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04,179.12,179.12,179.120000,179.12,1
949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15,72.20,72.20,72.200000,72.20,1
ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26,28.62,28.62,28.620000,28.62,1


In [91]:
report_df(orders_with_payments)

<class 'pandas.core.frame.DataFrame'>
Index: 99440 entries, e481f51cbdc54678b7cc49136f2d6af7 to 66dea50a8b16d9b4dee7af250b4be1a5
Data columns (total 12 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   customer_id                    99440 non-null  object        
 1   order_status                   99440 non-null  object        
 2   order_purchase_timestamp       99440 non-null  datetime64[ns]
 3   order_approved_at              99280 non-null  object        
 4   order_delivered_carrier_date   97657 non-null  object        
 5   order_delivered_customer_date  96475 non-null  datetime64[ns]
 6   order_estimated_delivery_date  99440 non-null  datetime64[ns]
 7   max_pay                        99440 non-null  float64       
 8   min_pay                        99440 non-null  float64       
 9   mean_pay                       99440 non-null  float64       
 10  total_pay                    

In [92]:
orders_with_payments['last_purchase'] = orders_with_payments['order_purchase_timestamp'].max()  # ULTIMA COMPRA
# Calcula la fecha y hora de la última compra en todo el dataset y la asigna a una nueva columna llamada 'last_purchase'.
# Es importante notar que todas las filas tendrán el *mismo* valor para 'last_purchase', que es la última fecha de compra de *todo* el dataset.

orders_with_payments['time_since_last_purchase'] =\
orders_with_payments['last_purchase'] - orders_with_payments['order_purchase_timestamp']  # TIEMPO DESDE ULTIMA COMPRA
# Calcula el tiempo transcurrido desde la última compra para cada orden.
# Resta la fecha de compra de cada orden ('order_purchase_timestamp') de la fecha de la última compra ('last_purchase').
# El resultado es un objeto timedelta que representa la diferencia en tiempo.

orders_with_payments['delivery_time'] =\
orders_with_payments['order_delivered_customer_date'] - orders_with_payments['order_purchase_timestamp']  # TIEMPO DE ENTREGA
# Calcula el tiempo de entrega real para cada orden.
# Resta la fecha de compra de la fecha de entrega al cliente.
# El resultado es un objeto timedelta.

orders_with_payments['delay'] =\
orders_with_payments['order_delivered_customer_date'] - orders_with_payments['order_estimated_delivery_date']  # RETRASO EN ENTREGA
# Calcula el retraso (o adelanto) en la entrega para cada orden.
# Resta la fecha de entrega estimada de la fecha de entrega real.
# El resultado es un objeto timedelta.

In [93]:
orders_with_payments.head(3).T

order_id,e481f51cbdc54678b7cc49136f2d6af7,53cdb2fc8bc7dce0b6741e2150273451,47770eb9100c2d0c44946d9cf07ec65d
customer_id,9ef432eb6251297304e76186b10a928d,b0830fb4747a6c6d20dea0b8c802d7ef,41ce2a54c0b03bf3443c3d931a367089
order_status,delivered,delivered,delivered
order_purchase_timestamp,2017-10-02 10:56:33,2018-07-24 20:41:37,2018-08-08 08:38:49
order_approved_at,2017-10-02 11:07:15,2018-07-26 03:24:27,2018-08-08 08:55:23
order_delivered_carrier_date,2017-10-04 19:55:00,2018-07-26 14:31:00,2018-08-08 13:50:00
order_delivered_customer_date,2017-10-10 21:25:13,2018-08-07 15:27:45,2018-08-17 18:06:29
order_estimated_delivery_date,2017-10-18 00:00:00,2018-08-13 00:00:00,2018-09-04 00:00:00
max_pay,18.59,141.46,179.12
min_pay,2.0,141.46,179.12
mean_pay,12.903333,141.46,179.12


<a id = "join"></a>
# Join final con clientes y variables finales
[Volver al índice](#table_of_contents)

In [94]:
df_final = pd.merge(customer_df, orders_with_payments, on = 'customer_id')

# Combina los DataFrames 'customer_df' y 'orders_with_payments' en un nuevo DataFrame llamado 'df_final' utilizando la columna 'customer_id' como clave para la unión.
# Esto une la información de los clientes con la información de sus órdenes y pagos.

In [95]:
df_final.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,max_pay,min_pay,mean_pay,total_pay,max_seq,last_purchase,time_since_last_purchase,delivery_time,delay
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP,delivered,2017-05-16 15:05:35,2017-05-16 15:22:12,2017-05-23 10:47:57,2017-05-25 10:35:35,2017-06-05,146.87,146.87,146.87,146.87,1,2018-10-17 17:30:18,519 days 02:24:43,8 days 19:30:00,-11 days +10:35:35
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP,delivered,2018-01-12 20:48:24,2018-01-12 20:58:32,2018-01-15 17:14:59,2018-01-29 12:41:19,2018-02-06,335.48,335.48,335.48,335.48,1,2018-10-17 17:30:18,277 days 20:41:54,16 days 15:52:55,-8 days +12:41:19
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP,delivered,2018-05-19 16:07:45,2018-05-20 16:19:10,2018-06-11 14:31:00,2018-06-14 17:58:51,2018-06-13,157.73,157.73,157.73,157.73,1,2018-10-17 17:30:18,151 days 01:22:33,26 days 01:51:06,1 days 17:58:51
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP,delivered,2018-03-13 16:06:38,2018-03-13 17:29:19,2018-03-27 23:22:42,2018-03-28 16:04:25,2018-04-10,173.30,173.30,173.30,173.30,1,2018-10-17 17:30:18,218 days 01:23:40,14 days 23:57:47,-13 days +16:04:25
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP,delivered,2018-07-29 09:51:30,2018-07-29 10:10:09,2018-07-30 15:16:00,2018-08-09 20:55:48,2018-08-15,252.25,252.25,252.25,252.25,1,2018-10-17 17:30:18,80 days 07:38:48,11 days 11:04:18,-6 days +20:55:48


In [96]:
df_final.set_index('customer_unique_id', inplace = True)

# Establece la columna 'customer_unique_id' del DataFrame 'df_final' como el índice del DataFrame.
# 'inplace = True' modifica el DataFrame directamente, sin necesidad de asignarlo a una nueva variable.

In [97]:
df_final.head()

,customer_id,customer_zip_code_prefix,customer_city,customer_state,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,max_pay,min_pay,mean_pay,total_pay,max_seq,last_purchase,time_since_last_purchase,delivery_time,delay
customer_unique_id,,,,,,,,,,,,,,,,,,,
861eff4711a542e4b93843c6dd7febb0,06b8999e2fba1a1fbc88172c00ba8bc7,14409,franca,SP,delivered,2017-05-16 15:05:35,2017-05-16 15:22:12,2017-05-23 10:47:57,2017-05-25 10:35:35,2017-06-05,146.87,146.87,146.87,146.87,1,2018-10-17 17:30:18,519 days 02:24:43,8 days 19:30:00,-11 days +10:35:35
290c77bc529b7ac935b93aa66c333dc3,18955e83d337fd6b2def6b18a428ac77,9790,sao bernardo do campo,SP,delivered,2018-01-12 20:48:24,2018-01-12 20:58:32,2018-01-15 17:14:59,2018-01-29 12:41:19,2018-02-06,335.48,335.48,335.48,335.48,1,2018-10-17 17:30:18,277 days 20:41:54,16 days 15:52:55,-8 days +12:41:19
060e732b5b29e8181a18229c7b0b2b5e,4e7b3e00288586ebd08712fdd0374a03,1151,sao paulo,SP,delivered,2018-05-19 16:07:45,2018-05-20 16:19:10,2018-06-11 14:31:00,2018-06-14 17:58:51,2018-06-13,157.73,157.73,157.73,157.73,1,2018-10-17 17:30:18,151 days 01:22:33,26 days 01:51:06,1 days 17:58:51
259dac757896d24d7702b9acbbff3f3c,b2b6027bc5c5109e529d4dc6358b12c3,8775,mogi das cruzes,SP,delivered,2018-03-13 16:06:38,2018-03-13 17:29:19,2018-03-27 23:22:42,2018-03-28 16:04:25,2018-04-10,173.30,173.30,173.30,173.30,1,2018-10-17 17:30:18,218 days 01:23:40,14 days 23:57:47,-13 days +16:04:25
345ecd01c38d18a9036ed96c73b8d066,4f2d8ab171c80ec8364f7c12e35b23ad,13056,campinas,SP,delivered,2018-07-29 09:51:30,2018-07-29 10:10:09,2018-07-30 15:16:00,2018-08-09 20:55:48,2018-08-15,252.25,252.25,252.25,252.25,1,2018-10-17 17:30:18,80 days 07:38:48,11 days 11:04:18,-6 days +20:55:48


In [98]:
df_final.columns.to_list()

# Obtiene una lista con los nombres de todas las columnas del DataFrame 'df_final'.
# 'df_final.columns': Accede al índice de las columnas del DataFrame.
# '.to_list()': Convierte el índice de las columnas en una lista de Python.

['customer_id',
 'customer_zip_code_prefix',
 'customer_city',
 'customer_state',
 'order_status',
 'order_purchase_timestamp',
 'order_approved_at',
 'order_delivered_carrier_date',
 'order_delivered_customer_date',
 'order_estimated_delivery_date',
 'max_pay',
 'min_pay',
 'mean_pay',
 'total_pay',
 'max_seq',
 'last_purchase',
 'time_since_last_purchase',
 'delivery_time',
 'delay']

In [99]:
lc = [
    'max_pay',
    'min_pay',
    'mean_pay',
    'total_pay',
    'max_seq',
    'time_since_last_purchase',
    'delivery_time',
    'delay'
]

# Crea una lista llamada 'lc' que contiene los nombres de varias columnas que probablemente representan características numéricas de los pagos y tiempos de entrega.
# Esta lista se utilizará probablemente para seleccionar o manipular estas columnas específicas en un DataFrame.

In [100]:
df_final = df_final[lc]

In [101]:
df_final.head()

,max_pay,min_pay,mean_pay,total_pay,max_seq,time_since_last_purchase,delivery_time,delay
customer_unique_id,,,,,,,,
861eff4711a542e4b93843c6dd7febb0,146.87,146.87,146.87,146.87,1,519 days 02:24:43,8 days 19:30:00,-11 days +10:35:35
290c77bc529b7ac935b93aa66c333dc3,335.48,335.48,335.48,335.48,1,277 days 20:41:54,16 days 15:52:55,-8 days +12:41:19
060e732b5b29e8181a18229c7b0b2b5e,157.73,157.73,157.73,157.73,1,151 days 01:22:33,26 days 01:51:06,1 days 17:58:51
259dac757896d24d7702b9acbbff3f3c,173.30,173.30,173.30,173.30,1,218 days 01:23:40,14 days 23:57:47,-13 days +16:04:25
345ecd01c38d18a9036ed96c73b8d066,252.25,252.25,252.25,252.25,1,80 days 07:38:48,11 days 11:04:18,-6 days +20:55:48


In [102]:
# Convierte la columna 'time_since_last_purchase' (que contiene diferencias de tiempo) a un número entero que representa el número de días.
df_final['time_since_last_purchase'] = df_final['time_since_last_purchase'].dt.days


In [103]:
# Convierte la columna 'delivery_time' (que contiene diferencias de tiempo) a un número entero que representa el número de días que tardó en entregar el pedido al cliente.
df_final['delivery_time'] = df_final['delivery_time'].dt.days

In [108]:
print(df_final['delivery_time'].dtype)

float64


In [106]:
# Convierte la columna 'delivery_time' (que contiene diferencias de tiempo) a un número entero que representa el número de días que tardó en entregar el pedido al cliente.
df_final['delivery_time'] = df_final['delivery_time'].dt.days



AttributeError: Can only use .dt accessor with datetimelike values

In [ ]:
df_final.head(3).T

In [ ]:
report_df(df_final)

In [ ]:
df_final.isnull().sum()

In [ ]:
df_final.head()

<a id = "skpipeline"></a>
# Creación de nuestros propios Transformers (CORE IDEA)
[Volver al índice](#table_of_contents)

Hasta ahora, siempre hemos utilizado los transformers de sklearn.

No obstante, en determinados casos nos podemos encontrar con la necesidad de definir nuestros propios transformers.

Existen 3 formas de hacer nuestros propias transformers:
1. Definirlos desde cero utilizando [BaseEstimator y TransformerMixin](https://towardsdatascience.com/customizing-sklearn-pipelines-transformermixin-a54341d8d624).
2. Modificar el comportamiento de uno existence para que se adapte a nuestro comportamiento.
3. Utilizar el FunctionTransformer de sklearn.

En este notebook vamos a usar el FunctionTransformer porque no requiere de conocimientos de [Programación Orientada a Objetos](https://en.wikipedia.org/wiki/Object-oriented_programming).

Para utilizar el FunctionTransformer primero tenemos que definir una función que haga "la transformación" que yo quiero.

En nuestro caso queremos agrupar las columnas por **customer_unique_id** para conseguir variables a nivel de cliente.

In [ ]:
def build_unique_id_features(X):

  aggreated_df = X.groupby(X.index).agg(
      n_orders = ('total_pay', 'count'),
      amount = ('total_pay', 'sum'),
      avg_ticket = ('total_pay', 'mean'),

      last_purchase = ('time_since_last_purchase', 'min'),
      first_purchase = ('time_since_last_purchase', 'max'),

      mean_delivery_time = ('delivery_time', 'mean'),
      max_delivery_time = ('delivery_time', 'max'),

      mean_delay = ('delay', 'mean'),
      max_delay = ('delay', 'max')
  )

  return aggreated_df

Una vez que tengo definida esta función, paso la función a FunctionTransformer y obtengo un Transformer 100% funcional.

In [ ]:
ClientIdGenerator = FunctionTransformer(func = build_unique_id_features)

Ahora podemos construir nuestro pipeline.

Los pasos que va a realizar son:

1. Usar el ***KNNImputer***, que se basa en la misma noción de **métricas de distancia** para imputar los valores nulos en función de los cliente más similares.

2. Creamos variables a nivel de cliente con ***ClientIdFeatureGenerator***. Nuestro DataFrame de salida será mas pequeño.

3. Estandarizamos los valores, usando ***RobustScaler*** para eliminar la influencia de los outliers.

4. Hacemos un fit con KMeans para calcular la **inertia** de los grupos (la dispersión de los datos al centroide).

Técnica del ***Elbow Curve***: todo esto lo hacemos en una loop porque queremos ver cuando hay un cambio brusco en la inertia. Dicho de otro modo **aumentar más el número de centroides no nos sale a cuenta porque la ganacia marginal es muy pequeña.**

**RECORDAD:** La inercia mide la suma de las distancias cuadradas entre cada punto de datos y el centroide del cluster al que pertenece. En otras palabras, cuantifica cuán compactos son los clusters.

In [ ]:
pipe = Pipeline(steps = [
    ('Imputer', KNNImputer()),
    ('CustomTransformer', ClientIdGenerator),
    ('RobustScaler', RobustScaler(quantile_range= (0, 99.0)))
])

In [ ]:
df_scaled_transformed = pipe.fit_transform(df_final)

In [ ]:
sse = {}

for k in range(2,15):

  print(f'Fitting pipe with {k} clusters')

  clustering_model = KMeans(n_clusters = k, random_state = 175)
  clustering_model.fit(df_scaled_transformed)

  sse[k] = clustering_model.inertia_


In [ ]:
sse

<a id = "elbow_curve"></a>
# Elbow curve (CORE IDEA)
[Volver al índice](#table_of_contents)

En esta sección vamos a visualizar nuestro ***Elbow Curve*** y buscaremos el punto de inflexión que será nuestro número de centroides.

In [ ]:
fig = plt.figure(figsize = (16, 8))
ax = fig.add_subplot()

x_values = list(sse.keys())
y_values = list(sse.values())

ax.plot(x_values, y_values, label = "Inertia/dispersión de los clústers")
fig.suptitle("Variación de la dispersión de los clústers en función de la k", fontsize = 16);

<a id = "segmentacion"></a>
# Segmentación de los clientes con la "k adecuada" (CORE IDEA)
[Volver al índice](#table_of_contents)

Ahora que hemos determinado el número de centroides correcto podemos fittear nuestro pipeline con la ***k adecuada.***

Dado que vamos a realizar nuestra segmentación con KMeans y vamos a suministrarle las variables de nuestro interés a veces a KMeans se le conoce como **segmentación no supervisada pero guiada**. Guiada porque de alguna manera el data scientist le dice (lo guía) a que discrimine usando unas variables y no otras.

In [ ]:
pipe = Pipeline(steps = [
    ('Imputer', KNNImputer()),
    ('CustomTransformer', ClientIdGenerator),
    ('RobustScaler', RobustScaler(quantile_range= (0, 99.0))),
    ('Clustering', KMeans(n_clusters = 5, random_state = 175))
])

In [ ]:
df_final.head()

In [ ]:
df_final.shape

In [ ]:
pipe.fit(df_final)

Una parte muy interesante de los pipelines es que la podemos filtrar (igual que una lista de python) y usar sólo parte de los pasos que tenemos implementados.

In [ ]:
pipe[:2]

In [ ]:
X_processed = pipe[:2].transform(df_final)

In [ ]:
X_processed.head()

In [ ]:
X_processed.shape

In [ ]:
labels = pipe.predict(df_final)

In [ ]:
X_processed['cluster'] = labels

In [ ]:
X_processed.head()

In [ ]:
X_processed[X_processed['cluster'] == 0]

<a id = "ficha"></a>
# Ficha de los clientes
[Volver al índice](#table_of_contents)

El último paso, usa vez que tenemos hecha nuestra segmentación completa es crear una ***ficha resumen*** de cada grupo con las principales variables de negocio o con aquellas que no se han utilizado en la segmentación para hacer un seguimiento periódico de los grupos o para enviar como documento al resto de los departamentos de la empresa.

In [ ]:
ficha_df = pd.DataFrame()

In [ ]:
for i, col in enumerate(['amount', 'n_orders', 'last_purchase', 'mean_delay']):

  resumen_data = X_processed[['cluster', col]].groupby('cluster').describe().T[1:]
  ficha_df = pd.concat([ficha_df, resumen_data])

In [ ]:
ficha_df

In [ ]:
out_index = [
    'Monetarios',
    'Fidelización',
    'Fidelización',
    'Logística'
]

inner_index = [
    'Importe',
    'Número de compras',
    'Última compra',
    'Retrasos'
]

estadisticos = ['Media', 'Desviación', 'Mínimo', 'Perc. 25', 'Perc. 50', 'Perc. 75', 'Máximo']

new_multiindex = []

for oi, ii in zip(out_index, inner_index):
  for es in estadisticos:
    new_multiindex.append((oi, ii, es))

In [ ]:
new_multiindex

In [ ]:
def generate_multiindex(list_of_tuples, names):
    return pd.MultiIndex.from_tuples(list_of_tuples, names = names)

In [ ]:
names = ['Grupo de indicadores', 'Indicadores', 'Estadísticos']
index_ficha = generate_multiindex(new_multiindex, names)
ficha_df.set_index(index_ficha, inplace = True)

In [ ]:
ficha_df

In [ ]:
tamaño_clusters = X_processed.groupby('cluster').size().to_frame().T

In [ ]:
tamaño_clusters.set_index(generate_multiindex([('General', 'Cluster', 'Tamaño')], names), inplace = True)

In [ ]:
tamaño_clusters

In [ ]:
ficha_df = pd.concat([tamaño_clusters, ficha_df])

In [ ]:
ficha_df

In [ ]:
ficha_df.style.background_gradient(cmap = 'Blues', axis = 1)

<a id = "conclusión"></a>
# Conclusión
[Volver al índice](#table_of_contents)

Los algoritmos no supervisados son herramientas **muy potentes** que cualquier data scientist debe tener a mano. Saber el funcionamiento y hacer la correcta implementación de estos puede permitir **extraer información muy valiosa y tomar mejores decisiones de negocio**. Hemos visto el algoritmo de KMeans que nos permiten crear **grupos homogéneos y accionables** de clientes y así mejorar los indicadores de la compañia.

Además de esto, hemos aprendido a utilizar los pipelines de sklearn para **automatizar al máximo el tratamiento de datos** así como implementar nuestros propios ***Transformers*** que puede ser utilizados dentro del pipeline.